In [4]:
def pasar_text(nombre,X,y):
    f = open(nombre,'a')
    h,w = X.shape
    for i in range(h):
        linea = str(y[i]+1) + ' '
        for j in range(w):
            linea = linea + str(j+1) + ':' + str(X[i,j]) + ' '
        #print(linea)
        f.write(linea + '\n')
    f.close()

In [16]:
pasar_text('clasificacionTrayNoDensasSinAumentar', np.array(X), np.array(y))

# Clasificacion Usando Trayectorias

In [6]:
import os
import subprocess
import numpy as np
import pandas as pd

i = 1
home = os.getenv("HOME")
videos = home + '/' + 'Vídeos/'
videos_pk = frames = os.listdir(videos + 'pk/')
videos_std = frames = os.listdir(videos + 'std/')

X = []
y = []

for video in videos_pk:
    subprocess.call(['bash', 'sacar_trayectorias_(tipo).sh', video,'pk/'])
    Trayectories = pd.read_csv('trayectories', sep=' ', header=None)
    del Trayectories[34]
    Trayectories = Trayectories[abs(Trayectories[32]-Trayectories[2])>=2]
    Trayectories = Trayectories[abs(Trayectories[33]-Trayectories[3])>=2]
    data = Trayectories.values
    X.append(data[:,2:])
    del Trayectories
    del data
    y.append(0)
    print('video: ' + str(i) + ' Procesado de: ' + str(len(videos_pk)+ len(videos_std)))
    i = i + 1

    
for video in videos_std:
    subprocess.call(['bash', 'sacar_trayectorias_(tipo).sh', video,'std/'])
    Trayectories = pd.read_csv('trayectories', sep=' ', header=None)
    del Trayectories[34]
    Trayectories = Trayectories[abs(Trayectories[32]-Trayectories[2])>=2]
    Trayectories = Trayectories[abs(Trayectories[33]-Trayectories[3])>=2]
    data = Trayectories.values
    X.append(data[:,2:])
    del Trayectories
    del data
    y.append(1)         
    print('video: ' + str(i) + ' Procesado de: ' + str(len(videos_pk)+ len(videos_std)))
    i = i + 1
    

video: 1 Procesado de: 14
video: 2 Procesado de: 14
video: 3 Procesado de: 14
video: 4 Procesado de: 14
video: 5 Procesado de: 14
video: 6 Procesado de: 14
video: 7 Procesado de: 14
video: 8 Procesado de: 14
video: 9 Procesado de: 14
video: 10 Procesado de: 14
video: 11 Procesado de: 14
video: 12 Procesado de: 14
video: 13 Procesado de: 14
video: 14 Procesado de: 14


In [7]:
for i in range(14):
    tray = X[i]
    total = tray.shape[0]
    mitad = int(total/2)
    inicio = mitad - 1500
    final = mitad + 1500
    data = tray[inicio:final]
    data = data.reshape((1,-1))
    X[i] = data[0]

In [8]:
for i in range(14):
    print(X[i].shape)

(96000,)
(96000,)
(96000,)
(96000,)
(96000,)
(96000,)
(96000,)
(96000,)
(96000,)
(96000,)
(96000,)
(96000,)
(96000,)
(96000,)


In [9]:
import numpy as np
print('x.shape: ', np.array(X).shape, 'y.shape: ', np.array(y).shape)

('x.shape: ', (14, 96000), 'y.shape: ', (14,))


In [5]:
from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score

# Naive Gaussian

In [74]:
from sklearn.naive_bayes import GaussianNB
est = GaussianNB()
i = 2
while i <=14:
    s = cross_val_score(est, X, y, cv=KFold(i, shuffle=True), scoring=make_scorer(accuracy_score))
    print (np.mean(s))
    i += 1



0.42857142857142855
0.2833333333333334
0.4375
0.6333333333333333
0.5555555555555555
0.5
0.4375
0.6111111111111112
0.55
0.45454545454545453
0.5
0.5
0.5


# DecisionTree

In [73]:
from sklearn.tree import DecisionTreeClassifier
est = DecisionTreeClassifier()
i = 2
while i <=14:
    s = cross_val_score(est, X, y, cv=KFold(i, shuffle=True), scoring=make_scorer(accuracy_score))
    print (np.mean(s))
    i += 1


0.7142857142857143
0.5666666666666668
0.41666666666666663
0.7333333333333333
0.8055555555555555
0.8571428571428571
0.75
0.7222222222222222
0.8
0.8181818181818182
0.7083333333333334
0.8076923076923077
0.8571428571428571


# Random Forest

In [75]:
from sklearn.ensemble import RandomForestClassifier
est = RandomForestClassifier()
i = 2
while i <=14:
    s = cross_val_score(est, X, y, cv=KFold(i, shuffle=True), scoring=make_scorer(accuracy_score))
    print (np.mean(s))
    i += 1


/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/estudiante/anaconda3/l

0.3571428571428571
0.4166666666666667
0.5208333333333333
0.3333333333333333
0.8888888888888888
0.7142857142857143
0.4375


/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/estudiante/anaconda3/l

0.7222222222222222
0.6
0.5909090909090909
0.6666666666666666
0.5384615384615384
0.5


/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/estudiante/anaconda3/l

# Support Vector Machine

In [77]:
from sklearn.svm import SVC
est = SVC()
i = 2
while i <=14:
    s = cross_val_score(est, X, y, cv=KFold(i, shuffle=True), scoring=make_scorer(accuracy_score))
    print (np.mean(s))
    i += 1


0.42857142857142855
0.35000000000000003
0.41666666666666663
0.13333333333333333
0.19444444444444442
0.35714285714285715
0.375
0.1111111111111111
0.05
0.045454545454545456
0.041666666666666664
0.0
0.0


/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/estudiante/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/estudiante/anaconda3/lib/python3.7/si

In [79]:
from sklearn.svm import SVC
est = SVC(kernel = 'linear')
i = 2
while i <=14:
    s = cross_val_score(est, X, y, cv=KFold(i, shuffle=True), scoring=make_scorer(accuracy_score))
    print (np.mean(s))
    i += 1


0.6428571428571428
0.6
0.49999999999999994
0.4
0.5555555555555555
0.5
0.5625
0.4444444444444444
0.4
0.45454545454545453
0.5416666666666666
0.46153846153846156
0.5


# K-Nearest Neighbors

In [92]:
from sklearn.neighbors import KNeighborsClassifier
est = KNeighborsClassifier(n_neighbors = 4)
i = 2
while i <=14:
    s = cross_val_score(est, X, y, cv=KFold(i, shuffle=True), scoring=make_scorer(accuracy_score))
    print (np.mean(s))
    i += 1


0.5714285714285714
0.5833333333333334
0.37499999999999994
0.6333333333333333
0.6666666666666666
0.5
0.5
0.6111111111111112
0.55
0.5909090909090909
0.6666666666666666
0.5384615384615384
0.5714285714285714


# Clasificar cada Trayectoria

In [36]:
import os
import subprocess
import numpy as np
import pandas as pd

i = 1
home = os.getenv("HOME")
videos = home + '/' + 'Vídeos/'
videos_pk = frames = os.listdir(videos + 'pk/')
videos_std = frames = os.listdir(videos + 'std/')
print videos_pk
print videos_std

X = []
labels = []

for video in videos_pk:
    
    subprocess.call(['bash', 'sacar_trayectorias_(tipo).sh', video,'pk/'])
    Trayectories = pd.read_csv('trayectories', sep=' ', header=None)
    del Trayectories[34]
    Trayectories = Trayectories[abs(Trayectories[32]-Trayectories[2])>=2]
    Trayectories = Trayectories[abs(Trayectories[33]-Trayectories[3])>=2]
    data = Trayectories.values
    
    for i in range(len(data)):
            
        X.append(data[i,2:])
        labels.append(0)
        
        

for video in videos_std:
    
    subprocess.call(['bash', 'sacar_trayectorias_(tipo).sh', video,'std/'])
    Trayectories = pd.read_csv('trayectories', sep=' ', header=None)
    del Trayectories[34]
    Trayectories = Trayectories[abs(Trayectories[32]-Trayectories[2])>=2]
    Trayectories = Trayectories[abs(Trayectories[33]-Trayectories[3])>=2]
    data = Trayectories.values
    
    for i in range(len(data)):
            
        X.append(data[i,2:])
        labels.append(1)


    
print 'X.SHAPE(): ', np.shape(X), 'y.shape: ', np.shape(labels) 

['pk1.mp4', 'pk3.mp4', 'pk5.mp4', 'pk4.mp4', 'pk7.mp4', 'pk2.mp4']
['std2.mp4', 'std4.mp4', 'std1.mp4', 'std3.mp4', 'std7.mp4', 'std6.mp4']
X.SHAPE():  (75967, 32) y.shape:  (75967,)


In [10]:
from sklearn.naive_bayes import GaussianNB

est = GaussianNB()
s = cross_val_score(est, X, y, cv=KFold(500, shuffle=True), scoring=make_scorer(accuracy_score))
print (np.mean(s))

0.572567347761796


In [13]:
from sklearn.ensemble import RandomForestClassifier
est = RandomForestClassifier()


s = cross_val_score(est, X, y, cv=KFold(20, shuffle=True), scoring=make_scorer(accuracy_score))
print (np.mean(s))

0.823545351433585


# Clasificar las trayectorias de un nuevo video

In [57]:
def Dibujar_rectas(frames_originales, path_originales, matriz_tray, Total_de_trayectorias, cantidad_de_fotogramas, y_kmeans):
    
    for i,frame in enumerate(frames_originales):
        
        img = cv2.imread(path_originales + frame)
        
        if i+1 != len(frames_originales):
            
            for j in range(Total_de_trayectorias):
                
                if(i+1>=matriz_tray[j,cantidad_de_fotogramas,0] and i+1<=matriz_tray[j,cantidad_de_fotogramas,1]):
                    
                    tray = matriz_tray[j,:i+2,:]
                    tray_mask = tray != 0
                    tray = tray[tray_mask]
                    w = tray.shape
                    w = int(w[0]/2)
                    tray = tray.reshape((w,2))
                    k = 0
                    intencidad = y_kmeans[j]
                    
                    while k <= w-2:
                        
                            if intencidad == 0:
                                
                                img = cv2.line(img,(int(tray[k,1]),int(tray[k,0])),(int(tray[k+1,1]),int(tray[k+1,0])),(255,0,0),1)
                                
                            else:
                                
                                img = cv2.line(img,(int(tray[k,1]),int(tray[k,0])),(int(tray[k+1,1]),int(tray[k+1,0])),(0,255,0),1)
                                

                                
                            k = k + 1
                            
        cv2.imwrite(path_originales + frame , img)

In [58]:
def sort_dir(path):
    frames = os.listdir(path)

    indices=[]

    for i in frames:
        indices.append(int(i[10:13]))


    sort_indices = np.sort(indices)
    sort_frames = []       
    for i in sort_indices:
            if i<10:
                sort_frames.append("fotograma-00"+str(i)+".png") 
            elif i<100:  
                 sort_frames.append("fotograma-0"+str(i)+".png")
            else:
                 sort_frames.append("fotograma-"+str(i)+".png")
    return sort_frames

In [39]:
def matriz_de_trayectorias(dataframe):
    matriz = np.zeros((Total_de_trayectorias, cantidad_de_fotogramas + 1, 2))
    data = dataframe.values
    matriz[:,cantidad_de_fotogramas,0] = data[:,0]
    matriz[:,cantidad_de_fotogramas,1] = data[:,1]
    for i in range(Total_de_trayectorias):
        tray = data[i,:].reshape((34,1))
        j = 0
        while j <= 32:
            if j == 0 or j == 1:
                inicio = int(tray[0])-1
            else:
                
                matriz[i,inicio,0] = tray[j]
                matriz[i,inicio,1] = tray[j+1]
                inicio = inicio + 1
            j = j + 2
        
    return matriz

In [47]:
subprocess.call(['bash', 'sacar_trayectorias_(tipo).sh', 'std5.mp4','std/'])

Trayectories = pd.read_csv('trayectories', sep=' ', header=None)
del Trayectories[34]
Tayectories = Trayectories[abs(Trayectories[32]-Trayectories[2])>=2]
Trayectories = Trayectories[abs(Trayectories[33]-Trayectories[3])>=2]
Trayectories_pk = Trayectories.values[:,2:]


Cabecera = pd.read_csv('cabecera', sep=' ', header=None)
Total_de_trayectorias = len(Trayectories[0])
y = Cabecera[1].values[0]
x = Cabecera[2].values[0]
cantidad_de_fotogramas = Cabecera[3].values[0]
print('Total de trayectorias: ' + str(Total_de_trayectorias), 'Alto: ' + str(y), 'Ancho: ' + str(x), 'Cantidad de fotogramas: ' + str(cantidad_de_fotogramas))

('Total de trayectorias: 11072', 'Alto: 270', 'Ancho: 480', 'Cantidad de fotogramas: 201')


In [59]:
import subprocess
from sklearn.ensemble import RandomForestClassifier


matriz_tray = matriz_de_trayectorias(Trayectories)
subprocess.call(['bash', 'sacar_fotogramas.sh', 'std', 'std5.mp4'])
frames = sort_dir('/home/experimentos/Documentos/repositorio_de_Ai/std/')
#print frames
est = RandomForestClassifier()
est.fit(X,labels)

y_pre = est.predict(Trayectories_pk)
#print y_pre
Dibujar_rectas(frames, '/home/experimentos/Documentos/repositorio_de_Ai/std/', matriz_tray, Total_de_trayectorias, cantidad_de_fotogramas, y_pre)